### Exercise 1
Suppose the people who own page 3 in the web of Figure 1 are infuriated by the
fact that its importance score, computed using formula (2.1), is lower than the score of page 1. In
an attempt to boost page 3’s score, they create a page 5 that links to page 3; page 3 also links to page
5. Does this boost page 3’s score above that of page 1?

![image](./image.png)